In [ ]:
!pip install --upgrade --quiet mdai

In [ ]:
import mdai
import pandas as pd
import numpy as np
import json
import shlex, subprocess

#### Update variables for your specific project

In [ ]:
project_id = '' # Get from Info tab in MD.ai UI
token = '' # Your private user token obtained from User Settings in MD.ai UI
domain = 'public.md.ai' #example

### Download annotations using API

In [ ]:
# Create Client to use library
mdai_client = mdai.Client(domain=domain, access_token=token)

Successfully authenticated to public.md.ai.


In [ ]:
# Download annotations
mdai_client.project(project_id, path='.',  annotations_only=True)

Using working directory for data.
Preparing annotations export for project MwBejRr1...                                                
Using cached annotations data for project MwBejRr1.
No project created. Downloaded annotations only.


In [ ]:
from pathlib import Path
from datetime import date

JSON = Path.cwd().glob("*.json")

# Convert annotation
r = {}
for j in JSON:
  if project_id in str(j) and date.today().strftime("%Y-%m-%d") in str(j):
    r = mdai.common_utils.json_to_dataframe(j)
    print(j)
    break
  
# or get from UI and enter json filename
if len(r) == 0:
  j = 'mdai_annotations.json' #enter json filename
  r = mdai.common_utils.json_to_dataframe(j)


/content/mdai_public_project_MwBejRr1_annotations_labelgroup_all_2020-08-28-170509.json


### Dataframes

In [ ]:
annotations = r['annotations']
a = annotations
l = r['labels']

## Progress

In [ ]:
class Manager:     
    def __init__(self, dataset, labelgroup, deadline, user_df, annots_df):
        self.dataset = dataset
        self.labelgroup = labelgroup
        self.deadline = deadline
        self.df = user_df 
        self.annots_df = annots_df
        self.user_dict = dict(zip(user_df.userId, user_df.name))
    
    #parse exam list
    def parse_exam_list(self, exam_str):
        if exam_str == -1:
            return []
        exams = []

        if exam_str != None:
            exam_str = str(exam_str)
            exam_str = exam_str.split(',')
            for g in exam_str:
                parts = g.split('-')
                if len(parts)>1:
                    exams.extend(range(int(parts[0]),int(parts[1])+1))
                else:
                    exams.append(int(g))
        return exams

    def get_done_exams(self, user_annots):
        #set conditions for being done here
        return user_annots.number.unique()

    def is_done_assignments(self, assignment_df):
        df = assignment_df.copy()
        df = df[df.labelGroupRestriction == self.labelgroup]

        df['goal'] = 0  
        df['done'] = 0
        #df['todo'] = 0
        #df['percent_done'] = '0%'  
        df['ontime'] = 0
        df['done_total'] = 0
        df['dataset'] = self.dataset
        df['exams_todo'] = ''

        for i, row in df.iterrows():
            exam_goals = self.parse_exam_list(row['assigned'])
            user_annots = self.annots_df[(self.annots_df.createdById == row.userId) & (self.annots_df.datasetId == self.dataset)]
            total_exams_done = self.get_done_exams(user_annots)
            ontime = user_annots[user_annots.createdAt <= self.deadline].number.unique()
            done = len(set(exam_goals).intersection(set(total_exams_done)))


            df.loc[i,'done'] = done
            df.loc[i,'done_total'] = len(total_exams_done)
            df.loc[i,'goal'] = len(exam_goals)
            # if len(exam_goals) > 0:
            #     df.loc[i,'percent_done'] = str(int(done/len(exam_goals) * 100))+'%'
            #df.loc[i,'todo'] = max(0,len(exam_goals)-done)
            todo = set(exam_goals).difference(set(total_exams_done))
            threshold = 10
            if len(todo) >= 0 and len(todo) <= threshold:
                #print(todo)
                df.loc[i,'exams_todo'] = str(list(todo))
            elif exam_goals == []:
                df.loc[i,'exams_todo'] = '-'
            else:
                df.loc[i,'exams_todo'] = 'Too many to list'
            df.loc[i,'ontime'] = len(ontime)

        return df


    #assignment dictionary from json
    def get_assignments(self, dicomStudyRestriction):
        if dicomStudyRestriction != -1:
            for d in dicomStudyRestriction:
                if d['dicomDatasetId'] == self.dataset:
                    return d['dicomDatasetStudyNumbers']
        return -1


    def get_all_assignments(self):
        df = self.df.copy()
        #expand_assignments 
        df['assigned'] = df.dicomStudyRestriction.apply(lambda o: self.get_assignments(o))
        df = df.drop('dicomStudyRestriction',axis=1)
        df = df[df['assigned'].notna()]

        return df


    def check_progress(self):
        df = self.get_all_assignments()
        return self.is_done_assignments(df).sort_values('done', ascending = False) 

### Download CLI tool

In [ ]:
# download CLI, get latest version number from https://docs.md.ai/cli/installation/
zip_file = 'mdai_cli_0.18.1_linux_amd64.zip'

In [ ]:
!wget -q -O {zip_file} https://s3.amazonaws.com/mdai-packages/cli/{zip_file}
!unzip -o {zip_file}
!rm -f {zip_file}
!mv mdai /usr/local/bin/

Archive:  mdai_cli_0.18.0_linux_amd64.zip
  inflating: mdai                    


### Get users and assignments with CLI

In [ ]:
user_output = 'users.json'
assign_output = 'assignments.json'
cmds = [{'command': f'mdai config domain {domain}', 'out':'output.txt'},
        {'command':f'mdai config token {token}', 'out':'output.txt'},
        {'command':f'mdai project list-users --project-id {project_id} --json > {user_output}','out':user_output},
        {'command':f'mdai project list-user-assignments --project-id {project_id} --json > {assign_output}','out':assign_output}]
    
def run_cli_cmds(cmds):
    error = open('error.txt', "w+", encoding="utf8") # create output file
    output = open('tmp.txt', "w+", encoding="utf8")
    try:
        for c in cmds:
            print(c['command'])
            out = c['out']
            output = open(out, "w+", encoding="utf8") # create output file
            subprocess.run(shlex.split(c['command']), stdout=output, stderr=error, check=True)
    except Exception:
        print("exception")

    #close file
    output.close()  
    error.close()
    
run_cli_cmds(cmds)

mdai config domain public.md.ai
mdai config token 28da3bc159992c276d836fd966ddfc97
mdai project list-users --project-id MwBejRr1 --json > users.json
mdai project list-user-assignments --project-id MwBejRr1 --json > assignments.json


In [ ]:
#get assignments for users with reader or writer role, adjust if only get writers' progress
with open(user_output) as f:
    content = json.load(f)

users = pd.DataFrame(content['data'])
writer_ids = users[(users.role == 'READER') | (users.role == 'WRITER')].id.unique()

with open(assign_output) as f:
    content = json.load(f)
df = pd.DataFrame(content['data']).fillna(-1)
df.labelGroupRestriction = df.labelGroupRestriction.astype('int')
user_df = df[df.userId.isin(writer_ids)]

In [ ]:
user_df

# Get Progress

In [ ]:
#subclass Manager class and define conditions for being done

class MyProjectManager(Manager):
    def get_done_exams(self, user_annots):
        #set conditions for being done here
        return user_annots.number.unique()
    
   
#sample variables
deadline = '2020-12-05'
#get all assigned labelgroups
labelgroups = user_df.labelGroupRestriction.unique()
#get all datasets
datasets = a.datasetId.unique()

results = pd.DataFrame()

# loop through each dataset and labelgroup
for dataset in datasets:
  for labelgroup in labelgroups:    
    mgr = MyProjectManager(dataset, labelgroup, deadline, user_df, a)
    results = results.append(mgr.check_progress())
#only show the results for users with assignments, comment out to see all
results = results[results.goal > 0]
results

,userId,name,labelGroupRestriction,assigned,goal,done,todo,percent_done,ontime,done_total,dataset,exams_todo
3,U_w8dP8D,aorta@aiai.cool,1.0,1-2,2,0,2,0%,0,0,D_PQZmo6,"[1, 2]"


## Email

### Note: this may not work in Colab. Download this notebook and use on your local machine to email progress.

In [ ]:
import smtplib, ssl

port = 465  # For SSL

# Create a secure SSL context
context = ssl.create_default_context()

smtp_server = "smtp.gmail.com"
sender_email = ""  # Enter your address
receiver_email = ""  # Enter receiver address
password = input("Type your password and press enter: ")
subject = 'Subject: MD.ai Project progess.'
intro = 'Project {project_id} progress.'

#send result from Manager
message = f"""\ {subject} {intro} {result}"""

with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)
